In [284]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim

import json
import requests

# Web Scraping

In [285]:
from bs4 import BeautifulSoup

wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

response = requests.get(wiki_url)

soup = BeautifulSoup(response.text, 'lxml')

table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df['Neighborhood'].replace(' / ', ', ', inplace=True, regex=True) 
df = df.reset_index(drop=True)

# df[ df['Neighborhood'] == '' ] # There are no case with blank Neighborhood

df.head()
#df = df.astype({'Postal code':'String','Borough':'String' ,'Neighborhood':'String'})

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [286]:
df.shape

(103, 3)

In [287]:
df.groupby('Postal code')
df.shape

(103, 3)

# Get the latitude and the longitude coordinates

In [288]:
geolocation_file = 'Geospatial_Coordinates.csv'
gdf = pd.read_csv('https://cocl.us/Geospatial_data')
gdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Add Latitude and Longitude

In [289]:
df['Latitude'] = None
df['Longitude'] = None
df = df.astype({'Postal code':'str','Borough':'str' ,'Neighborhood':'str', 'Latitude':'float64','Longitude':'float64'})
df.dtypes

Postal code      object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [290]:
gdf = gdf.astype({'Postal Code':'str','Latitude':'float64','Longitude':'float64'})
gdf.rename(columns={'Postal Code': 'Postal code'},inplace = True)
df.dtypes

Postal code      object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [291]:
lat = list()
long = list()
for i, row in df.iterrows():
    key = df.iloc[i]['Postal code']
    for j, jrow in gdf.iterrows():
        if key == gdf.iloc[j]['Postal code']:
            lat.append( gdf.iloc[j]['Latitude'] )
            long.append( gdf.iloc[j]['Longitude'])


df['Latitude'] = lat
df['Longitude'] = long
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Neighborhoods in Toronto

In [292]:
df = df[df.Borough.str.contains('Toronto', case=False)]

In [268]:
df = df.reset_index(drop=True)
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [293]:
map_tor = folium.Map(location=[43.686412, -79.400049], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

# Using Foursquare API to get Venues and Cluster

In [294]:
CLIENT_ID = 'RENETLD1TJZCQUM420LSQRHFLQEPAJWA4RUXA22TNQMORHYG'
CLIENT_SECRET = 'E2T0ZHMPK2KKI2WUWELG3I0YZKACXV2SNMQG0HXKJDSXSJFS'
VERSION = '20180604'
LIMIT = 30

In [295]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [296]:
tnt_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
print(tnt_venues.shape)
tnt_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [297]:
# one hot encoding
tnt_onehot = pd.get_dummies(tnt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tnt_onehot['Neighborhood'] = tnt_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tnt_onehot.columns[-1]] + list(tnt_onehot.columns[:-1])
tnt_onehot = tnt_onehot[fixed_columns]

tnt_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [298]:
tnt_onehot.shape

(863, 188)

In [299]:
tnt_grouped = tnt_onehot.groupby('Neighborhood').mean().reset_index()
tnt_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.033333,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [300]:
tnt_grouped.shape

(39, 188)

In [301]:
num_top_venues = 5

for hood in tnt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tnt_grouped[tnt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0            Beer Bar  0.07
1         Coffee Shop  0.07
2        Cocktail Bar  0.07
3  Seafood Restaurant  0.07
4         Fish Market  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0       Nightclub  0.12
1            Café  0.12
2     Coffee Shop  0.08
3  Breakfast Spot  0.08
4             Gym  0.04


----Business reply mail Processing CentrE----
              venue  freq
0        Smoke Shop  0.06
1     Auto Workshop  0.06
2              Park  0.06
3       Pizza Place  0.06
4  Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4       Coffee Shop  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.23
1                Café  0.07
2  Italian Restaurant  

In [302]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [303]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tnt_grouped['Neighborhood']

for ind in np.arange(tnt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tnt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Coffee Shop,Cocktail Bar,Beer Bar,Café,Basketball Stadium,Bistro,Restaurant,Breakfast Spot,Jazz Club
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Bar,Bakery,Stadium,Burrito Place,Pet Store,Performing Arts Venue
2,Business reply mail Processing CentrE,Garden,Auto Workshop,Garden Center,Park,Comic Shop,Pizza Place,Light Rail Station,Burrito Place,Recording Studio,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Coffee Shop,Sculpture Garden,Boat or Ferry,Bar,Boutique,Airport
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Falafel Restaurant,Middle Eastern Restaurant,Comic Shop,Park,Ramen Restaurant,Bubble Tea Shop


In [304]:
# set number of clusters
kclusters = 5

tnt_grouped_clustering = tnt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tnt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1])

In [305]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [306]:
tnt_merged = df
tnt_merged = tnt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
tnt_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Breakfast Spot,Theater,Bakery,Yoga Studio,Restaurant,Pub,Café,Performing Arts Venue
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Sandwich Place,Distribution Center,Discount Store,Juice Bar,Beer Bar,Burger Joint
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Café,Coffee Shop,Theater,Burger Joint,Shopping Mall,Hotel,Diner,Restaurant,Ramen Restaurant,Plaza
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Gastropub,Café,Coffee Shop,Gym,Jazz Club,Restaurant,Italian Restaurant,Diner,Ice Cream Shop,Hotel
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Trail,Pub,Health Food Store,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


In [307]:
# create map
map_clusters = folium.Map(location=[43.651494, -79.375418], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tnt_merged['Latitude'], tnt_merged['Longitude'], tnt_merged['Neighborhood'], tnt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters